# Inverting image operations

In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy.sparse as sparse
import scipy.sparse.linalg as spla
%matplotlib inline

# https://matplotlib.org/users/customizing.html
# print(plt.style.available) # uncomment to print all styles
import seaborn as sns
sns.set(font_scale=2)
plt.style.use('seaborn-whitegrid')
mpl.rcParams['figure.figsize'] = (10.0, 8.0)

## Import an image of a Social Security Number

In [ ]:
from PIL import Image
img = Image.open('ssn.png')
xmat = (255 - np.asarray(img).max(axis=2))/255
x = xmat.flatten()
plt.imshow(xmat);

## Construct a "blur" matrix

In [ ]:
def setup_filter_parameters(rmin,nelx,nely):
    # Filter: Build (and assemble) the index+data vectors for the coo matrix format
    nfilter=int(nelx*nely*((2*(np.ceil(rmin)-1)+1)**2))
    iH = np.zeros(nfilter)
    jH = np.zeros(nfilter)
    sH = np.zeros(nfilter)
    cc=0
    for i in range(nelx):
        for j in range(nely):
            row=i*nely+j
            kk1=int(np.maximum(i-(np.ceil(rmin)-1),0))
            kk2=int(np.minimum(i+np.ceil(rmin),nelx))
            ll1=int(np.maximum(j-(np.ceil(rmin)-1),0))
            ll2=int(np.minimum(j+np.ceil(rmin),nely))
            for k in range(kk1,kk2):
                for l in range(ll1,ll2):
                    col=k*nely+l
                    fac=rmin-np.sqrt(((i-k)*(i-k)+(j-l)*(j-l)))
                    iH[cc]=row
                    jH[cc]=col
                    sH[cc]=np.maximum(0.0,fac/rmin)
                    cc=cc+1
    # Finalize assembly and convert to csc format
    H=sparse.coo_matrix((sH,(iH,jH)),shape=(nelx*nely,nelx*nely)).tocsc()
    return H

rmin = 5
nelx = xmat.shape[0]
nely = xmat.shape[1]
As = setup_filter_parameters(rmin,nelx,nely)

## Compute b = A x
### b is the blurred image

In [ ]:
b = As @ x

In [ ]:
plt.imshow(b.reshape(xmat.shape));

## Assume we have the blurred image, solve for the unblurred one

In [ ]:
import scipy.linalg as sla
A = As.todense()
P, L, U = sla.lu(A)

### Let's look at L and U

In [ ]:
plt.figure()
plt.subplot(131)
plt.spy(A); plt.axis('off')

plt.subplot(132)
plt.spy(L); plt.axis('off')

plt.subplot(133)
plt.spy(U); plt.axis('off')

### $A = L * U$ right?

In [ ]:
np.max(A - np.dot(L, U))

### Acutally $A = P * L * U$

In [ ]:
np.max(A - np.dot(P, np.dot(L, U)))

### What is $P$ 

It's called a "permutation" matrix...from pivoting.

### What about "Solving"?

If $Ax = P L U x = b$, then there are two steps:
1. $y \leftarrow \text{solve}\,\, L y = P^Tb$
2. $x \leftarrow \text{solve}\,\, U x = y$

In [ ]:
y = sla.solve_triangular(L, np.dot(P.T, b), lower=True)
x_solve = sla.solve_triangular(U, y)

In [ ]:
plt.imshow(x_solve.reshape(xmat.shape))

## Who cares? 

Why not just `np.linalg.solve`?

Let's time two things
1. factorization
2. solving, given a factorization

In [ ]:
%timeit sla.solve(A, b)

In [ ]:
%timeit sla.solve_triangular(U, y)